<a href="https://colab.research.google.com/github/deep1185/ga-learner-dsmp-repo/blob/master/fashion_mnist_deepmandal1185_17thnov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,565,050
Trainable params: 1,565,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 403s 239ms/step - loss: 0.1469 - accuracy: 0.9451 - val_loss: 0.2572 - val_accuracy: 0.9110
Epoch 5/10
1688/1688 [==============================] - 404s 239ms/step - loss: 0.1117 - accuracy: 0.9583 - val_loss: 0.2709 - val_accuracy: 0.9145
Epoch 6/10
1688/1688 [==============================] - 404s 239ms/step - loss: 0.0824 - accuracy: 0.9689 - val_loss: 0.3093 - val_accuracy: 0.9092
Epoch 7/10
1688/1688 [==============================] - 404s 239ms/step - loss: 0.0602 - accuracy: 0.9768 - val_loss: 0.3735 - val_accuracy: 0.9095
Epoch 8/10
1688/1688 [==============================] - 412s 244ms/step - loss: 0.0466 - accuracy: 0.9819 - val_loss: 0.3931 - val_accuracy: 0.9067
Epoch 9/10
1688/1688 [==============================] - 416s 247ms/step - loss: 0.0364 - accuracy: 0.9872 - val_loss: 0.4232 - val_accuracy: 0.9080
Epoch 10/10
1688/1688 [==============================] - 409s 242ms/step - loss: 0.0326 - accuracy: 0.9884 - val